In [1]:
import os
import fiona
import hashlib
import boto3
import json
import datacube
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
from madmex.models import SegmentationInformation
from madmex.util import parser_extra_args
from madmex.settings import TEMP_DIR
from importlib import import_module
import gc

In [2]:
name_of_product = 's2_10m_fiona_test_2_2018'
gwf_kwargs = {'region': 'Jalisco', 'begin': '2018-01-01', 'end':'2018-12-31'}
gwf_kwargs.update(product=name_of_product)

In [3]:
iterable = gwf_query(**gwf_kwargs)

In [4]:
lista_iter = list(iterable)

In [5]:
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(28, -27),
 (28, -28),
 (30, -29),
 (30, -30),
 (28, -29),
 (28, -30),
 (29, -30),
 (30, -27),
 (29, -29),
 (30, -28),
 (29, -28),
 (29, -27)]

In [31]:
var = lista_iter[0]

In [32]:
algorithm = 'slic' #'slic'
datasource = 'sentinel2'
name = 's2_10m_some_scenes_fiona_test'
year = '2018'
extra_args = parser_extra_args(['n_segments=10000', 'compactness=0.5']) #for slic: parser_extra_args(['n_segments=10000', 'compactness=0.5'])

In [33]:
band_list = ['ndvi_mean']

In [34]:
try:
    module = import_module('madmex.segmentation.%s' % algorithm)
    Segmentation = module.Segmentation
except ImportError as e:
    raise ValueError('Invalid model argument')

In [35]:
meta, _ = SegmentationInformation.objects.get_or_create(
    algorithm=algorithm, datasource=datasource,
    parameters=json.dumps(extra_args),
    datasource_year=year,
    name=name)

In [36]:
meta

<SegmentationInformation: SegmentationInformation object (1)>

In [37]:
geoarray = GridWorkflow.load(var[1], measurements=band_list)

In [38]:
dataset_name = meta.datasource + '_%d_%d_' % (var[0][0], var[0][1]) + meta.datasource_year

In [39]:
dataset_name

'sentinel2_28_-27_2018'

In [40]:
hash = hashlib.md5(dataset_name.encode('utf-8')).hexdigest()[0:6]

In [41]:
name_file = hash + '_' + dataset_name

In [42]:
name_file

'a6cd39_sentinel2_28_-27_2018'

In [43]:
path = os.path.join(TEMP_DIR, 'segmentation_results')

In [44]:
path

'/shared_volume/temp/segmentation_results'

In [45]:
if not os.path.exists(path):
    os.makedirs(path)

In [46]:
seg = Segmentation.from_geoarray(geoarray, **extra_args)

In [47]:
seg.geobox.extent.wkt

'POLYGON ((2377720 1126200,2377720 1076180,2427740 1076180,2427740 1126200,2377720 1126200))'

In [48]:
seg.geobox.extent.json

{'type': 'Polygon',
 'coordinates': [[(2377720.0, 1126200.0),
   (2377720.0, 1076180.0),
   (2427740.0, 1076180.0),
   (2427740.0, 1126200.0),
   (2377720.0, 1126200.0)]]}

In [49]:
try:
    seg.segment()
    seg.polygonize(crs_out=None)  #None for no reprojection to latlon
    seg.to_filesystem(path,name_file)
    seg.to_db(name_file, meta)
    seg.to_bucket(path, name_file)
    gc.collect()
except Exception as e:
        print(e)